In [1]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [2]:
path_cats_and_dogs = f"{getcwd()}/tmp/cats-and-dogs.zip"

local_zip = path_cats_and_dogs
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./tmp')
zip_ref.close()

In [3]:
print(len(os.listdir('./tmp/PetImages/Cat/')))
print(len(os.listdir('./tmp/PetImages/Dog/')))

1500
1500


In [4]:
os.mkdir('./cats-v-dogs/training/cats')
os.mkdir('./cats-v-dogs/training/dogs')
os.mkdir('./cats-v-dogs/testing/cats')
os.mkdir('./cats-v-dogs/testing/cats')

FileNotFoundError: [Errno 2] No such file or directory: './cats-v-dogs/training/cats'